In [ ]:
#import ALL THE THINGS

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import os
import feather
import datetime

from ema_workbench import (Model, 
                           RealParameter, 
                           Constant,
                           TimeSeriesOutcome,
                           perform_experiments, 
                           ema_logging, 
                           save_results,
                           perform_experiments, 
                           ema_logging)

#from ema_workbench.em_framework import samplers

from ema_workbench.connectors.vensim import VensimModel

#from ema_workbench.em_framework.evaluators import LHS, MC

import ema_workbench.analysis.pairs_plotting as pairs
import ema_workbench.analysis.plotting as emaplt

# turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

C:\Users\Patrick\Anaconda3\lib\site-packages\ema_workbench\em_framework\optimization.py:29: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)


In [ ]:
def export_outcomes(outcomes, location = './Data/' ):
    """takes EMA outcomes and exports all time series for each outcome of interest as a .feather dataframe for easy
    import into R. Arguments: export_outcomes(name of EMA outcomes dict, desired save location). Save location must exist,
    defaults to './Data/'.
    
    """
     
    today = datetime.date.today()
    datestr = (str(today))  
    
    keylist = list(outcomes.keys())
    
    for k in keylist:
        df_temp = pd.DataFrame(outcomes[k])
        df_temp = df_temp.copy()
        path = location + datestr + k.replace(' ','') + '.feather'
        feather.write_dataframe(df_temp,path)

    return("Done.");

## Values and response surface for r, k
<img src="StrogatzBudworms_rk_combined.PNG" alt="r, k from Strogatz" style="width: 700px;"/>

In [ ]:
#define experiment parameters
uncertainties = [
                #RealParameter('k', 2,30), #dimensionless carrying capacity for budworms #fixed for now TODO
                RealParameter('r max',0.4,0.41), #initial r value
                RealParameter('drift',0.3,0.35) #amount by which r is reduced at time "step time"
            ]

outcomes =  [
                TimeSeriesOutcome('x'), #budworm population
                TimeSeriesOutcome('c'), #budworm creation
                TimeSeriesOutcome('p') #budworm predation/loss
            ]

constants = [
                Constant('xi', 2), #initial budworm population
                Constant('k', 10), #dimensionless carrying capacity for budworms
                Constant('use rsin',1), #use step function for r instead of constant base value
                Constant('r base val',0.4), #constant base value for r
                
                Constant('drift time',40), #fixed time at which drift of r sets in, has no effect on system behavior
                Constant('drift duration',20) #time it takes r to drift from max to min
                
            ]

In [ ]:
wd = r'./'
model = VensimModel("StrogatzBudworms", wd=wd, model_file=r'201805310123_BBSD_BW_StrogatzBudworms_rk_rsinsweep.vpm')

model.uncertainties = uncertainties
model.outcomes = outcomes
model.constants = constants

In [ ]:

#run experiments
#TODO parallel
results = perform_experiments(model, 200)

experiments, outcomes = results

In [ ]:
figure = emaplt.lines(results,density=u'kde') #show lines, and end state density
plt.rcParams["figure.figsize"] = (18,10)
plt.show() #show figure

In [ ]:
#experiments, outcomes = results
#print(experiments) #<class 'numpy.ndarray'>
#print(type(experiments.item(2))) #<class 'tuple'>

#experiments is an array of tuples, where each tuple is one experiment input set

In [ ]:
#print(type(outcomes)) #<class 'dict'>
#print(type(outcomes['Deceased Population'])) #<class 'numpy.ndarray'>
#print(type(outcomes['Deceased Population'][1])) #<class 'numpy.ndarray'>
#print(type(outcomes['Deceased Population'][1].item(1))) #<class 'float'>


'''outcomes is a dict of output variables (key) and output value arrays (value), where each output value array has
the shape (number of runs) * (run duration / time step = total steps). Each run (or row) is an array of floats,
where each row is the time series output of a specific simulation run for the associated variable, and each float
is the value of that variable at a specific step or time in the simulation run.'''


In [ ]:
export_outcomes(outcomes)

In [ ]:
print(experiments)

In [ ]:
df_expt = pd.DataFrame(experiments)
df_expt = df_expt.copy()
df_expt.iloc[:,0:2].head()

In [ ]:
import datetime
df_expt = pd.DataFrame(experiments) #convert experiments (array of tuples) into df
df_expt = df_expt.copy() #self-copy to circumvent errors
path = "./Data/" + str(datetime.date.today()) + "Experiments" + '.feather' #create path for saving
feather.write_dataframe(df_expt.iloc[:,0:2],path) #feather it!